Hello

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('http://projects.fivethirtyeight.com/general-model/president_general_polls_2016.csv')
df.head() #display the first five rows of dataframe

,cycle,branch,type,matchup,forecastdate,state,startdate,enddate,pollster,grade,...,adjpoll_clinton,adjpoll_trump,adjpoll_johnson,adjpoll_mcmullin,multiversions,url,poll_id,question_id,createddate,timestamp
0,2016,President,polls-plus,Clinton vs. Trump vs. Johnson,11/8/16,U.S.,11/3/2016,11/6/2016,ABC News/Washington Post,A+,...,45.20163,41.72430,4.626221,NaN,NaN,https://www.washingtonpost.com/news/the-fix/wp...,48630,76192,11/7/16,09:35:33 8 Nov 2016
1,2016,President,polls-plus,Clinton vs. Trump vs. Johnson,11/8/16,U.S.,11/1/2016,11/7/2016,Google Consumer Surveys,B,...,43.34557,41.21439,5.175792,NaN,NaN,https://datastudio.google.com/u/0/#/org//repor...,48847,76443,11/7/16,09:35:33 8 Nov 2016
2,2016,President,polls-plus,Clinton vs. Trump vs. Johnson,11/8/16,U.S.,11/2/2016,11/6/2016,Ipsos,A-,...,42.02638,38.81620,6.844734,NaN,NaN,http://projects.fivethirtyeight.com/polls/2016...,48922,76636,11/8/16,09:35:33 8 Nov 2016
3,2016,President,polls-plus,Clinton vs. Trump vs. Johnson,11/8/16,U.S.,11/4/2016,11/7/2016,YouGov,B,...,45.65676,40.92004,6.069454,NaN,NaN,https://d25d2506sfb94s.cloudfront.net/cumulus_...,48687,76262,11/7/16,09:35:33 8 Nov 2016
4,2016,President,polls-plus,Clinton vs. Trump vs. Johnson,11/8/16,U.S.,11/3/2016,11/6/2016,Gravis Marketing,B-,...,46.84089,42.33184,3.726098,NaN,NaN,http://www.gravispolls.com/2016/11/final-natio...,48848,76444,11/7/16,09:35:33 8 Nov 2016


In [3]:
print("Number of rows (polls): " + str(df.shape[0]))
print("Number of columns (categories): " + str(df.shape[1]))
print("\nNumber of empty values for each column:")
print(df.isnull().sum())

Number of rows (polls): 12624
Number of columns (categories): 27

Number of empty values for each column:
cycle                   0
branch                  0
type                    0
matchup                 0
forecastdate            0
state                   0
startdate               0
enddate                 0
pollster                0
grade                1287
samplesize              3
population              0
poll_wt                 0
rawpoll_clinton         0
rawpoll_trump           0
rawpoll_johnson      4227
rawpoll_mcmullin    12534
adjpoll_clinton         0
adjpoll_trump           0
adjpoll_johnson      4227
adjpoll_mcmullin    12534
multiversions       12588
url                     3
poll_id                 0
question_id             0
createddate             0
timestamp               0
dtype: int64


In [4]:
categories = ['type', 'state', 'enddate', 'pollster', 'grade', 'samplesize', 'population',
             'adjpoll_clinton', 'adjpoll_trump', 'adjpoll_johnson', 'adjpoll_mcmullin', 'poll_id']
df2 = df.loc[:, categories]
df2.head()

,type,state,enddate,pollster,grade,samplesize,population,adjpoll_clinton,adjpoll_trump,adjpoll_johnson,adjpoll_mcmullin,poll_id
0,polls-plus,U.S.,11/6/2016,ABC News/Washington Post,A+,2220.0,lv,45.20163,41.72430,4.626221,NaN,48630
1,polls-plus,U.S.,11/7/2016,Google Consumer Surveys,B,26574.0,lv,43.34557,41.21439,5.175792,NaN,48847
2,polls-plus,U.S.,11/6/2016,Ipsos,A-,2195.0,lv,42.02638,38.81620,6.844734,NaN,48922
3,polls-plus,U.S.,11/7/2016,YouGov,B,3677.0,lv,45.65676,40.92004,6.069454,NaN,48687
4,polls-plus,U.S.,11/6/2016,Gravis Marketing,B-,16639.0,rv,46.84089,42.33184,3.726098,NaN,48848


Hello

In [5]:
print(df2.loc[:,'type'].unique())

['polls-plus' 'now-cast' 'polls-only']


HI

In [6]:
df_po = df2[df2.loc[:,'type']=='polls-only']
df_po = df_po.reset_index(drop=True) 
df_po.head()

,type,state,enddate,pollster,grade,samplesize,population,adjpoll_clinton,adjpoll_trump,adjpoll_johnson,adjpoll_mcmullin,poll_id
0,polls-only,U.S.,11/6/2016,ABC News/Washington Post,A+,2220.0,lv,45.21947,41.70754,4.606925,NaN,48630
1,polls-only,U.S.,11/7/2016,Google Consumer Surveys,B,26574.0,lv,43.40083,41.14659,5.164047,NaN,48847
2,polls-only,U.S.,11/6/2016,Ipsos,A-,2195.0,lv,42.01984,38.74365,6.816055,NaN,48922
3,polls-only,U.S.,11/7/2016,YouGov,B,3677.0,lv,45.68214,40.90047,6.118311,NaN,48687
4,polls-only,U.S.,11/6/2016,Gravis Marketing,B-,16639.0,rv,46.83107,42.27754,3.749071,NaN,48848


In [7]:
df_po.describe()

,samplesize,adjpoll_clinton,adjpoll_trump,adjpoll_johnson,adjpoll_mcmullin,poll_id
count,4207.000000,4208.000000,4208.000000,2799.000000,30.000000,4208.000000
mean,1148.216068,43.322517,42.654425,4.651088,24.508827,45910.899477
std,2630.856265,7.097772,6.948612,2.472390,5.235812,2864.763228
min,35.000000,17.115890,4.488276,-3.677883,11.028320,35362.000000
25%,447.500000,40.220230,38.449348,3.130344,23.108497,45151.750000
50%,772.000000,44.142125,42.704720,4.366810,25.135225,46384.500000
75%,1236.500000,46.901398,46.315503,5.763004,27.976062,47741.250000
max,84292.000000,86.713200,72.376610,20.357000,31.574690,48922.000000


Hi

In [8]:
df_po.loc[:,'enddate'].head()

0    11/6/2016
1    11/7/2016
2    11/6/2016
3    11/7/2016
4    11/6/2016
Name: enddate, dtype: object

Hey

In [9]:
df_po.loc[:,'enddate'] = pd.to_datetime(df_po.loc[:,'enddate']) #convert 'enddate' into 'datetime' variables
df_po.loc[:, 'enddate'].head()

0   2016-11-06
1   2016-11-07
2   2016-11-06
3   2016-11-07
4   2016-11-06
Name: enddate, dtype: datetime64[ns]

In [10]:
df_po.loc[:, ['enddate', 'adjpoll_clinton', 'adjpoll_trump', 'adjpoll_johnson', 'adjpoll_mcmullin']].head(10)

,enddate,adjpoll_clinton,adjpoll_trump,adjpoll_johnson,adjpoll_mcmullin
0,2016-11-06,45.21947,41.70754,4.606925,NaN
1,2016-11-07,43.40083,41.14659,5.164047,NaN
2,2016-11-06,42.01984,38.74365,6.816055,NaN
3,2016-11-07,45.68214,40.90047,6.118311,NaN
4,2016-11-06,46.83107,42.27754,3.749071,NaN
5,2016-11-06,49.05626,43.87898,3.018706,NaN
6,2016-11-06,45.31196,40.80614,4.230162,NaN
7,2016-11-05,43.68695,40.80897,5.381917,NaN
8,2016-11-06,45.03026,41.83415,8.034579,NaN
9,2016-11-07,42.88452,42.18602,6.367243,NaN


In [11]:
df_po.loc[:,'grade'].unique()

array(['A+', 'B', 'A-', 'B-', 'A', nan, 'B+', 'C+', 'C-', 'C', 'D'], dtype=object)

Hi